In [20]:
import openai
openai.api_key = OPENAI_API_KEY

response = openai.Embedding.create(
  model="text-embedding-ada-002",
  input="The food was delicious and the waiter..."
)

emb = response['data'][0]['embedding']
print(len(emb))

1536


In [23]:
# print(emb)

In [24]:
## REMEMBER TO DELETE THIS LINE BEFORE UPLOADING
api_key = COHERE_API_KEY

In [25]:
import cohere  
co = cohere.Client(api_key)  
texts = [  
   'Hello from Cohere!'
]  
response = co.embed(texts=texts, model='embed-english-light-v2.0')  
embeddings = response.embeddings # All text embeddings 
print(len(embeddings[0])) # Print embeddings for the first text

1024


In [26]:
import pinecone      

pinecone.init(      
	api_key=PINECONE_API_KEY,      
	environment='us-west1-gcp'      
)      
index = pinecone.Index('pokemon-cards-v2')

/home/ec3dev/personal/github/pyenv/pte/lib/python3.8/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [27]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [29]:
pinecone.describe_index("pokemon-cards-v2")

IndexDescription(name='pokemon-cards-v2', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1.x1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [30]:
from datasets import load_dataset

In [31]:
pcds = load_dataset("bhavnicksm/PokemonCardsPlus", split='train')
pcds

Generating train split: 100%|██████████| 13139/13139 [00:00<00:00, 185515.20 examples/s]


Dataset({
    features: ['id', 'name', 'card_image', 'pokemon_image', 'caption', 'pokemon_intro', 'pokedex_text', 'hp', 'set_name', 'blip_caption'],
    num_rows: 13139
})

In [32]:
pcds = pcds.map(lambda example: {"text" : f"Name: {example['name']}\nSet Name: {example['set_name']}\nPokemon Caption: {example['blip_caption']}\nCard Description: {example['caption']}\nPokemon Description: {example['pokemon_intro']}\nPokedex Entry: {example['pokedex_text']}"})

Map: 100%|██████████| 13139/13139 [00:01<00:00, 8815.29 examples/s]


In [33]:
pcds

Dataset({
    features: ['id', 'name', 'card_image', 'pokemon_image', 'caption', 'pokemon_intro', 'pokedex_text', 'hp', 'set_name', 'blip_caption', 'text'],
    num_rows: 13139
})

In [35]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

In [36]:
pcds = pcds.map(lambda example : {"length" : len(enc.encode(example['text']))})

Map: 100%|██████████| 13139/13139 [00:05<00:00, 2254.06 examples/s]


In [39]:
# It costs less than $1
sum(list(pcds['length'])) * (0.0001/1000)

0.8565002

In [77]:
pcds = pcds.map(lambda example : {"pokemon_image" : example['pokemon_image'] if example['pokemon_image'] != None else ''})
pcds = pcds.map(lambda example : {"pokemon_intro" : example['pokemon_intro'] if example['pokemon_intro'] != None else ''})
pcds = pcds.map(lambda example : {"pokedex_text" : example['pokedex_text'] if example['pokedex_text'] != None else ''})
pcds = pcds.map(lambda example : {"blip_caption" : example['blip_caption'] if example['blip_caption'] != None else ''})

Map: 100%|██████████| 13139/13139 [00:01<00:00, 9823.60 examples/s] 


In [78]:
from time import time, sleep

In [79]:
from torch.utils.data import DataLoader

In [87]:
dl = DataLoader(pcds, batch_size=64)

In [81]:
# emb = co.embed(texts=texts, model='embed-english-light-v2.0')

In [88]:
pinecone_obj = {'id': None, 'values': None, 'metadata' : None}

In [113]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 13139}},
 'total_vector_count': 13139}

In [90]:
from tqdm import tqdm, trange

In [91]:
for batch in tqdm(dl):
    
    # print(batch)
    
    # text = batch['text']
    # response = openai.Embedding.create(
    # model="text-embedding-ada-002",
    # input=texts
    # )
    # emb = response['data'][0]['embedding']

    # upsert_obj = pinecone_obj.copy()

    # upsert_obj['id'] = batch['id']
    # upsert_obj['values'] = emb
    # upsert_obj['metadata'] = {"img_url": batch['card_image'],
    #                     "pokimg_url" : batch['pokemon_image'] if batch['pokemon_image'] != None else '', 
    #                     "name" : batch['name'], 
    #                     "description": batch['caption'],
    #                     "pokemon_intro": batch['pokemon_intro'] if batch['pokemon_intro'] != None else '', 
    #                     "pokedex_entry": batch['pokedex_text'] if batch['pokedex_text'] != None else '', 
    #                     "blip_caption" : batch['blip_caption'] if batch['blip_caption'] != None else '',  
    #                     "hp": int(batch['hp'])}
    
    # upsert_resp = index.upsert(vectors=[upsert_obj])


    upsert_list = []
    texts = batch['text']

    response = openai.Embedding.create(
    model="text-embedding-ada-002",
    input=texts
    )

    # # print(len(response['data']))
    embs = [x['embedding'] for x in response['data']]
    # # print(len(embs))

    for i in range(len(batch['id'])):
        pcd = pinecone_obj.copy()
        pcd['id'] = batch['id'][i]
        pcd['values'] = embs[i]
        pcd['metadata'] = {"img_url": batch['card_image'][i],
                           "pokimg_url" : batch['pokemon_image'][i] if batch['pokemon_image'][i] != None else '', 
                           "name" : batch['name'][i], 
                           "description": batch['caption'][i],
                           "pokemon_intro": batch['pokemon_intro'][i] if batch['pokemon_intro'][i] != None else '', 
                           "pokedex_entry": batch['pokedex_text'][i] if batch['pokedex_text'][i] != None else '', 
                           "blip_caption" : batch['blip_caption'][i] if batch['blip_caption'][i] != None else '',  
                           "hp": int(batch['hp'][i])}
        upsert_list.append(pcd)
    
    # print(len(upsert_list))
    # print(upsert_list[0]['values'])
    
    upsert_resp = index.upsert(vectors=upsert_list)

    # break

100%|██████████| 206/206 [34:11<00:00,  9.96s/it]  


In [84]:
# index.delete(delete_all=True)

{}

In [92]:
query = "sad"

response = openai.Embedding.create(
                                model="text-embedding-ada-002",
                                input=query
                                )

query_emb = response['data'][0]['embedding']
query_emb

[0.00135776586830616,
 -0.01316700130701065,
 0.0049084085039794445,
 -0.026983262971043587,
 -0.01668599247932434,
 0.009440246038138866,
 -0.023009788244962692,
 -0.019283033907413483,
 0.00495385704562068,
 -0.03056718036532402,
 0.020049160346388817,
 0.02345128543674946,
 0.005989427212625742,
 -0.010738766752183437,
 -0.010764737613499165,
 0.024386221542954445,
 0.054641757160425186,
 -0.010173910297453403,
 0.025165332481265068,
 -0.027502670884132385,
 -0.004337059333920479,
 0.002937903394922614,
 -0.005103186704218388,
 -0.0020825027022510767,
 -0.0031164500396698713,
 -0.003684552852064371,
 0.01268005557358265,
 -0.04051385074853897,
 0.014101935550570488,
 -0.015010900795459747,
 0.016257479786872864,
 -0.009985624812543392,
 -0.0327746644616127,
 -0.017854660749435425,
 -0.022347543388605118,
 -0.016335392370820045,
 -0.03602096810936928,
 -0.020477673038840294,
 -0.0074080610647797585,
 -0.013387749902904034,
 0.006888652686029673,
 0.002709039021283388,
 0.006206929218

In [114]:
query_response = index.query(query_emb, top_k=5, include_metadata=True)
query_response

{'matches': [{'id': 'mcd21-24',
              'metadata': {'blip_caption': 'a pokemon that looks like a bug '
                                           'with a big nose',
                           'description': 'A Basic Pokemon Card of type Water '
                                          'with the title Sobble and 60 HP of '
                                          "rarity None from the set McDonald's "
                                          'Collection 2021 and the flavor '
                                          'text: When scared, this Pokemon '
                                          'cries. Its tears pack the chemical '
                                          'punch of 100 onions, and attackers '
                                          "won't be able to resist weeping. It "
                                          'has the attack Bind with the cost '
                                          'Water, Colorless, the energy cost 2 '
                                 

In [99]:
response_texts = [q['metadata']['description'] for q in query_response['matches']]
response_texts

["A Basic Pokemon Card of type Water with the title Sobble and 60 HP of rarity None from the set McDonald's Collection 2021 and the flavor text: When scared, this Pokemon cries. Its tears pack the chemical punch of 100 onions, and attackers won't be able to resist weeping. It has the attack Bind with the cost Water, Colorless, the energy cost 2 and the damage of 20 with the description: Flip a coin. If heads, your opponent's Active Pokemon is now Paralyzed. It has weakness against Lightning 2. ",
 "A Basic Pokemon Card of type Water with the title Sobble and 60 HP of rarity Promo from the set SWSH Black Star Promos and the flavor text: When scared, this Pokemon cries. Its tears pack the chemical punch of 100 onions, and attackers won't be able to resist weeping. It has the attack Bind with the cost Water, Colorless, the energy cost 2 and the damage of 20 with the description: Flip a coin. If heads, your opponent's Active Pokemon is now Paralyzed. It has weakness against Lightning 2. ",

In [121]:
coresponse = co.rerank(model="rerank-english-v2.0", query=query, documents=response_texts, top_n=5)
reranked_index = [coresponse[i].index for i in range(5)]
reranked_index

[1, 0, 2, 4, 3]

In [126]:
query_response['matches']

ValueError: 0 is not in list

In [127]:
reranked_responses = [query_response['matches'][ind] for ind in reranked_index]

In [128]:
reranked_responses

[{'id': 'swshp-SWSH003',
  'metadata': {'blip_caption': 'a pokemon that looks like a bug with a big nose',
               'description': 'A Basic Pokemon Card of type Water with the '
                              'title Sobble and 60 HP of rarity Promo from the '
                              'set SWSH Black Star Promos and the flavor text: '
                              'When scared, this Pokemon cries. Its tears pack '
                              'the chemical punch of 100 onions, and attackers '
                              "won't be able to resist weeping. It has the "
                              'attack Bind with the cost Water, Colorless, the '
                              'energy cost 2 and the damage of 20 with the '
                              'description: Flip a coin. If heads, your '
                              "opponent's Active Pokemon is now Paralyzed. It "
                              'has weakness against Lightning 2. ',
               'hp': 60.0,
        

In [115]:
qque = query_response['matches'].copy()
reranked_objs = []
for i in range(len(query_response)):
    text = 

5

In [84]:
qurls = [x['metadata']['img_url'] for x in query_response['matches']]
qurls[0]

'https://images.pokemontcg.io/swshp/SWSH003_hires.png'

In [85]:
from IPython.display import Image, display
from IPython.core.display import HTML 

In [86]:
import ipyplot

In [87]:
# for url in qurls:
#     display(Image(url=url, width=200))

ipyplot.plot_images(qurls, img_width=200)